In [6]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import joblib
import os

os.makedirs("outputs", exist_ok=True)
print("Libraries loaded ✅")


Libraries loaded ✅


In [7]:
# Create a small messy dataset manually for demo
data = {
    "Age": [25, np.nan, 35, 40, np.nan, 30],
    "Salary": [50000, 60000, np.nan, 80000, 75000, 62000],
    "Department": ["HR", "Finance", "Finance", np.nan, "IT", "IT"]
}
df = pd.DataFrame(data)

print("[INFO] Raw Data:")
print(df)
df.to_csv("sample_data.csv", index=False)  # save sample


[INFO] Raw Data:
    Age   Salary Department
0  25.0  50000.0         HR
1   NaN  60000.0    Finance
2  35.0      NaN    Finance
3  40.0  80000.0        NaN
4   NaN  75000.0         IT
5  30.0  62000.0         IT


In [9]:
# Copy original for human-friendly cleanup
human_df = df.copy()

# Fill numeric NaNs with median
for col in human_df.select_dtypes(include=[np.number]).columns:
    median_val = human_df[col].median()
    human_df[col].fillna(median_val, inplace=True)

# Fill categorical NaNs with most frequent
for col in human_df.select_dtypes(include=['object']).columns:
    mode_val = human_df[col].mode()[0]
    human_df[col].fillna(mode_val, inplace=True)

# Drop duplicates (if any)
human_df.drop_duplicates(inplace=True)

print("[INFO] Cleaned Human Data:")
print(human_df)

# Save to outputs
human_df.to_csv("outputs/processed_data_human.csv", index=False)


[INFO] Cleaned Human Data:
    Age   Salary Department
0  25.0  50000.0         HR
1  32.5  60000.0    Finance
2  35.0  62000.0    Finance
3  40.0  80000.0    Finance
4  32.5  75000.0         IT
5  30.0  62000.0         IT


/tmp/ipython-input-2244172106.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  human_df[col].fillna(median_val, inplace=True)
/tmp/ipython-input-2244172106.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

In [10]:
numeric_features = human_df.select_dtypes(include=['int64','float64']).columns
categorical_features = human_df.select_dtypes(include=['object']).columns

# Numeric pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Categorical pipeline
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combine into preprocessor
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Fit & transform
processed_data = preprocessor.fit_transform(human_df)

# Get feature names
feature_names = preprocessor.get_feature_names_out()

# Convert to DataFrame
machine_df = pd.DataFrame(processed_data.toarray()
                          if hasattr(processed_data, 'toarray') else processed_data,
                          columns=feature_names)

print("[INFO] Machine-ready Data:")
print(machine_df.head())

# Save
machine_df.to_csv("outputs/processed_data_machine.csv", index=False)
joblib.dump(preprocessor, "outputs/preprocessor.joblib")


[INFO] Machine-ready Data:
   num__Age  num__Salary  cat__Department_Finance  cat__Department_HR  \
0 -1.643168    -1.492272                      0.0                 1.0   
1  0.000000    -0.486246                      1.0                 0.0   
2  0.547723    -0.285041                      1.0                 0.0   
3  1.643168     1.525806                      1.0                 0.0   
4  0.000000     1.022793                      0.0                 0.0   

   cat__Department_IT  
0                 0.0  
1                 0.0  
2                 0.0  
3                 0.0  
4                 1.0  


['outputs/preprocessor.joblib']

In [11]:
print("✅ Outputs generated in /outputs folder:")
!ls -lh outputs


✅ Outputs generated in /outputs folder:
total 12K
-rw-r--r-- 1 root root 3.9K Aug 24 16:03 preprocessor.joblib
-rw-r--r-- 1 root root  133 Aug 24 16:03 processed_data_human.csv
-rw-r--r-- 1 root root  359 Aug 24 16:03 processed_data_machine.csv


In [12]:
!zip -r Task1_ETL.zip task1_etl.ipynb sample_data.csv outputs


	zip warning: name not matched: task1_etl.ipynb
  adding: sample_data.csv (deflated 33%)
  adding: outputs/ (stored 0%)
  adding: outputs/processed_data_human.csv (deflated 40%)
  adding: outputs/preprocessor.joblib (deflated 62%)
  adding: outputs/processed_data_machine.csv (deflated 51%)
